In [1]:
import random

DECK_COMPLETE = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A',
                 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A',
                 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A',
                 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A']

BASIC_STRATEGY = [['H','D','D','D','D','H','H','H','H','H'],
                  ['D','D','D','D','D','D','D','D','H','H'],
                  ['D','D','D','D','D','D','D','D','D','H'],
                  ['H','H','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H'],
                  ['S','S','S','S','S','H','H','H','H','H']]

In [2]:
def rand_card(deck_func):
    card = random.choice(deck_func)
    deck_func.remove(card)
    return card

def score(cards):
    score = 0
    possible_hands = []
    aces = 0
    
    for x in cards:
        if type(x) == int:
            score += x
        elif x in ['J', 'Q', 'K']:
            score += 10
        else:
            score += 1
            aces += 1
            
    for i in range(aces+1):
        if (score + i * 10) <= 21:
            possible_hands.append(score + i * 10)

    if(possible_hands) == []:
        return -1
    return max(possible_hands)

In [3]:
#stand on 17
def bj_round():
    deck_current = DECK_COMPLETE.copy()
    
    player_cards = [rand_card(deck_func=deck_current), rand_card(deck_func=deck_current)]
    player_score = score(player_cards)
    dealer_cards = [rand_card(deck_func=deck_current)]
    dealer_score = score(dealer_cards)
    
    if(player_score==21):
        return ('Win', 21)
    
    game_state = 'Ongoing'
    strategy = ''
    double_flag = 0
    
    while game_state == 'Ongoing':
        if(player_score<=8):
            strategy = 'H'
        elif(player_score>=17):
            strategy = 'S'
        else:
            strategy = BASIC_STRATEGY[(player_score-9)][(dealer_score-2)]

        if(strategy=='D'):
            double_flag = 1
            player_cards.append(rand_card(deck_func=deck_current))
            player_score = score(player_cards)
            if(player_score==-1):
                game_state='Loss'
            else:
                game_state='S'
        if(strategy=='H'):
            player_cards.append(rand_card(deck_func=deck_current))
            player_score = score(player_cards)
            if(player_score==-1):
                game_state='Loss'
        if(strategy=='S'):
            game_state='S'
        
    if game_state == 'Loss':
        return ('Loss', player_score)
    
    while((dealer_score<17) ^ (dealer_score ==-1)):
        dealer_cards.append(rand_card(deck_func=deck_current))
        dealer_score = score(dealer_cards)
    
    if((player_score > dealer_score) or (dealer_score>21)):
        return ('Win', player_score)
    elif(player_score == dealer_score):
        return ('Draw', player_score)
    else:
        return ('Loss', player_score)

In [4]:
results={'Win':[], 'Draw':[], 'Loss':[]}

In [5]:
for _ in range(1000000):
    wdl, total = bj_round()
    results[wdl].append(total)

In [6]:
from collections import Counter

for wasd in results.values():
    print("##############")
    
    total=dict(Counter(wasd))
    
    sorted_dict = dict(sorted(total.items(), reverse=True))
    
    for key, value in sorted_dict.items():
        print(f"{key}:{round(value*100/sum(sorted_dict.values()),3)}%")      

##############
21:21.937%
20:25.144%
19:14.813%
18:11.416%
17:7.892%
16:3.709%
15:3.962%
14:4.039%
13:4.224%
12:2.784%
11:0.08%
##############
21:7.96%
20:32.822%
19:18.313%
18:19.564%
17:21.341%
##############
20:4.026%
19:7.492%
18:11.185%
17:15.426%
16:5.302%
15:5.659%
14:5.687%
13:5.996%
12:3.798%
11:0.107%
-1:35.322%


In [7]:
def bj_round_till_win(cum_loss=0):
    deck_current = DECK_COMPLETE.copy()
    
    player_cards = [rand_card(deck_func=deck_current), rand_card(deck_func=deck_current)]
    player_score = score(player_cards)
    dealer_cards = [rand_card(deck_func=deck_current)]
    dealer_score = score(dealer_cards)
    
    if(player_score==21):
        return ('Win', 21, 21 + cum_loss)
    
    game_state = 'Ongoing'
    strategy = ''
    double_flag = 0
    
    while game_state == 'Ongoing':
        if(player_score<=8):
            strategy = 'H'
        elif(player_score>=17):
            strategy = 'S'
        else:
            strategy = BASIC_STRATEGY[(player_score-9)][(dealer_score-2)]

        if(strategy=='D'):
            player_cards.append(rand_card(deck_func=deck_current))
            player_score = score(player_cards)
            if(player_score==-1):
                cum_loss += 1
                game_state='L'
            else:
                cum_loss += -1
                game_state='S'
        if(strategy=='H'):
            player_cards.append(rand_card(deck_func=deck_current))
            player_score = score(player_cards)
            if(player_score==-1):
                game_state='L'
        if(strategy=='S'):
            game_state='S'
        
    if game_state == 'Loss':
        return bj_round_till_win(cum_loss=cum_loss-2)
    
    while((dealer_score<17) ^ (dealer_score ==-1)):
        dealer_cards.append(rand_card(deck_func=deck_current))
        dealer_score = score(dealer_cards)
    
    if((player_score > dealer_score) or (dealer_score>21)):
        return ('Win', player_score, player_score + cum_loss)
    elif(player_score == dealer_score):
        return bj_round_till_win(cum_loss=cum_loss)
    else:
        return bj_round_till_win(cum_loss=cum_loss-2)
        
bj_round_till_win()

('Win', 18, 14)

In [15]:
#min score 7
for _ in range(4):
    player = []
    for _ in range(6):
        player.append(max(bj_round_till_win()[2]-3,7))
    print(sorted(player, reverse=True))

[18, 16, 15, 14, 14, 10]
[17, 17, 14, 12, 11, 7]
[17, 15, 14, 13, 8, 7]
[17, 16, 16, 13, 7, 7]


In [42]:
def bj_round_till_win_2(cum_loss=0):
    deck_current = DECK_COMPLETE.copy()
    
    player_cards = [rand_card(deck_func=deck_current), rand_card(deck_func=deck_current)]
    player_score = score(player_cards)
    dealer_cards = [rand_card(deck_func=deck_current)]
    dealer_score = score(dealer_cards)
    
    if(player_score==21):
        return ('Win', 21, 21 - cum_loss)
    
    game_state = 'Ongoing'
    strategy = ''
    double_flag = 0
    
    while game_state == 'Ongoing':
        if(player_score<=8):
            strategy = 'H'
        elif(player_score>=17):
            strategy = 'S'
        else:
            strategy = BASIC_STRATEGY[(player_score-9)][(dealer_score-2)]

        if(strategy=='D'):
            player_cards.append(rand_card(deck_func=deck_current))
            player_score = score(player_cards)
            if(player_score==-1):
                cum_loss += 1
                game_state='L'
            else:
                cum_loss += -1
                game_state='S'
        if(strategy=='H'):
            player_cards.append(rand_card(deck_func=deck_current))
            player_score = score(player_cards)
            if(player_score==-1):
                game_state='L'
        if(strategy=='S'):
            game_state='S'
        
    if game_state == 'Loss':
        return ('L', cum_loss-1)
    
    while((dealer_score<17) ^ (dealer_score ==-1)):
        dealer_cards.append(rand_card(deck_func=deck_current))
        dealer_score = score(dealer_cards)
    
    if((player_score > dealer_score) or (dealer_score>21)):
        return ('Win', cum_loss, player_score - cum_loss)
    elif(player_score == dealer_score):
        return ('D', cum_loss)
    else:
        return ('L', cum_loss+1)
        
bj_round_till_win_2()

('Win', 0, 20)

In [18]:
#min score 7
for _ in range(4):
    cum_loss=0
    player = []
    while(len(player)<6):
        
        game = bj_round_till_win_2(cum_loss)
        cum_loss = game[1]
        
        if(game[0]=='Win'):            
            player.append(max(game[2]-3,7))
        
    print(sorted(player, reverse=True))

[18, 18, 17, 11, 10, 10]
[17, 14, 14, 13, 12, 7]
[17, 13, 12, 12, 11, 7]
[19, 17, 17, 16, 11, 9]
